In [203]:
import pandas as pd
import plotly
import dash

df_deputados = pd.read_csv("dados/deputados.csv", sep=';')
df_deputados.drop(["uri", "nome", "nomeCivil", "cpf", "municipioNascimento", "dataNascimento", "dataFalecimento", "urlRedeSocial", "urlWebsite", "idLegislaturaFinal", "ufNascimento"], axis=1, inplace=True)
df_deputados = df_deputados.query("idLegislaturaInicial >= 22")
df_deputados = df_deputados.set_axis(["idLegislatura", "sexo"], axis=1, inplace=False)
df_deputados["count"] = 1;

df_legislaturas = pd.read_csv("dados/legislaturas.csv", sep=';')
df_legislaturas.drop(["uri", "dataInicio", "dataFim"], axis=1, inplace=True)
df_legislaturas = df_legislaturas.query("idLegislatura >= 22")

In [204]:
df_deputados

,idLegislatura,sexo,count
1151,42,M,1
1152,46,M,1
1153,48,M,1
1154,40,M,1
1155,48,M,1
...,...,...,...
7586,56,M,1
7587,56,F,1
7588,56,M,1
7589,56,M,1


In [317]:
df = df_deputados.merge(df_legislaturas, on="idLegislatura")
df

,idLegislatura,sexo,count,anoEleicao
0,42,M,1,1962
1,42,M,1,1962
2,42,M,1,1962
3,42,M,1,1962
4,42,F,1,1962
...,...,...,...,...
6378,51,M,1,1998
6379,51,M,1,1998
6380,51,M,1,1998
6381,51,M,1,1998


In [318]:
df_deputadas_F = pd.DataFrame({
    "anoEleicao": [1890, 1893, 1896, 1899, 1902, 1905, 1908, 1911, 1914, 1917, 1920, 1923, 1926, 1929, 1945, 1958, 1970],
    "sexo": ["F", "F", "F", "F", "F", "F", "F", "F", "F", "F", "F", "F", "F", "F", "F", "F", "F"],
    "count": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    }
)

In [319]:
df = df.groupby(["anoEleicao", "sexo"], as_index=False).count()
df

,anoEleicao,sexo,idLegislatura,count
0,1890,M,206,206
1,1893,M,135,135
2,1896,M,81,81
3,1899,M,84,84
4,1902,M,77,77
5,1905,M,71,71
6,1908,M,66,66
7,1911,M,86,86
8,1914,M,70,70
9,1917,M,75,75


In [320]:
df.drop(["idLegislatura"], axis=1, inplace=True)
df = pd.concat([df, df_deputadas_F])
df.sort_values(by=['anoEleicao'], ascending=True, inplace=True)
df.reset_index(inplace=True)
df.drop(["index"], axis=1, inplace=True)

In [321]:
df

,anoEleicao,sexo,count
0,1890,M,206
1,1890,F,0
2,1893,M,135
3,1893,F,0
4,1896,M,81
...,...,...,...
65,2010,F,29
66,2014,M,212
67,2014,F,38
68,2018,M,234


In [322]:
import plotly.express as px

fig = px.area(
    df,
    title="Número de novos deputados em cada ano por sexo",
    x="anoEleicao",
    y="count",
    # nbins=45,
    color="sexo",
    color_discrete_map={
        "M": 'rgb(99, 110, 250)',
        "F": 'rgb(239, 85, 59)'
    },
)
fig.show()

In [374]:
y_m = df.query("sexo == 'M'")["count"]
y_f = df.query("sexo == 'F'")["count"]
x = df["anoEleicao"].unique()

In [393]:
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x, 
    y=y_f, 
    fill='tonexty',
    fillcolor='rgba(255, 60, 49, 0.5)',
    mode='none',
    name="Mulheres"
))

fig.add_trace(go.Scatter(
    x=x,
    y=y_m,
    fill='tonexty',
    fillcolor='rgba(25, 97, 251, 0.5)',
    mode='none',
    name="Homens"
))

fig.update_layout(
    title_text="Quantidade de NOVOS deputados por sexo em cada eleição",
    # title_x=0.5
)

fig.show()